Experiment with OpenCV for finding the pool table edges within an image.

This version assumes a full-sized vertical image of the table. It is looking
for the marker pattern along the side of the table.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
def ImgShow(images, resolution=120):
    plt.rcParams['figure.dpi'] = resolution
    
    cnt = len(images)
    if cnt > 20:
        # Top-level object is an image, make it an array of 1
        images = [images]
        cnt = len(images)
        
    cols = cnt
    fig, ax = plt.subplots(1, cnt)    
    if 1 == cnt:
        ax = [ax]
    for i in range(0, len(images)):
        dims = len(images[i].shape)
        if 3 == dims:
            imgDisp = cv.cvtColor(images[i], cv.COLOR_BGR2RGB)
        else:
            imgDisp = cv.cvtColor(images[i], cv.COLOR_GRAY2RGB)
            
        ax[i].axis('off')
        ax[i].imshow(imgDisp)

In [ ]:
# Load the test image
imageOriginal = cv.imread("../test/images/long_all.jpg")

In [ ]:
#Find the edges using flood, start with a blur
blur = int(7)
imageBlur = cv.GaussianBlur(imageOriginal, (blur,blur), cv.BORDER_DEFAULT)

# Edge detect using canny
#imageCannyMarksOnly = cv.imread("../test/images/CannyMarksOnly.jpg")
#ret, imageCanny = cv.threshold(imageCannyMarksOnly, 127, 255, 0)
imageCanny = cv.Canny(imageBlur, 100, 175)

ImgShow([imageBlur, imageCanny],220)

In [ ]:
# Contours
contours, b, = cv.findContours(imageCanny, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
imgDrawnContours = imageOriginal.copy()
cv.drawContours(imgDrawnContours, contours, -1, (0,255,0),3)
ImgShow([imageCanny,imgDrawnContours])

In [ ]:
# Find circles

class Circle:
    def __init__(self, x,y,r):
        self.x = x
        self.y = y
        self.r = r
        
    def center(self):
        return (self.x, self.y)
    
    def point(self):
        return (int(self.x+0.5), int(self.y+0.5))
    
    def radius(self):
        return self.r
    
circles = []
for points in contours:
    (x,y),radius = cv.minEnclosingCircle(points)
    circles.append(Circle(x,y,radius))
    
#sort by radius
circles.sort(key=lambda s: s.radius()) # sorts using lambda function

# print out the data
imgCircles = imageOriginal.copy()
clrs = []
clrs.append((255,0,0))
clrs.append((0,255,0))
clrs.append((0,0,255))
iClr = 0
for circle in circles:
    #print("{0}\t{1}\t{2}".format(circle.radius(), circle.center()[0], circle.center()[1]))
    center_coordinates = (int(circle.center()[0]+0.5), int(circle.center()[1]+0.5))
    cv.circle(imgCircles, center_coordinates, int(circle.radius()),clrs[iClr % 3])	
    iClr = iClr+1
    
    
ImgShow([imgCircles], 600)
#cv.imwrite("../test/images/dotz.jpg", imgCircles)

In [ ]:
# Find the lines that the dots line up on

class LineP:
    def __init__(self, x1,y1, x2,y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        rise = y2 - y1
        run = x2 - x1
        if run != 0.0:
            self.m = rise/run
        else:
            self.m = 10E10
        self.b = y1 - self.m * x1
        self.pointCount = 2
        
    def angleRad(self):
        return np.arctan(self.m)
    
    def angleDeg(self):
        return self.angleRad() * 180.0 / math.pi
    
    def length(self):
        rise = self.y2 - self.y1
        run = self.x2 - self.x1
        return math.sqrt(rise * rise + run * run)
    
    def pointCount(self):
        return self.pointCount
    def AddPoint(self):
        self.pointCount += 1
        
        
#Between every two points, there is a line. Find all of the possible lines.
lines = []
imgLines = imageOriginal.copy()
clr = (255,0,0)
angleLimitV = 20.0 # Only accept lines +/- this angle
angleLimitH = 5.0
for i in range(0, len(circles)-1):
    for j in range(i+1, len(circles)):
        circle_a = circles[i]
        circle_b = circles[j]
        line = LineP(circle_a.center()[0],circle_a.center()[1], circle_b.center()[0],circle_b.center()[1])
        angle = line.angleDeg()
        if(angle > -angleLimitH and angle < angleLimitH or angle > 90-angleLimitV and angle < 90+angleLimitV or angle > -90-angleLimitV and angle < -90+angleLimitV):
            lines.append(line)
            #print(line.angleDeg())
            cv.line(imgLines, circle_a.point(), circle_b.point(), clr)
        
ImgShow([imgLines],400)
